In [ ]:
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Preprocessing data

In [ ]:
data_dir = pathlib.Path('/tmp/data').with_suffix('')
training_ds = tf.keras.utils.image_dataset_from_directory(
    directory=data_dir,
    color_mode='grayscale',
    validation_split=0.1,
    subset='training',
    image_size=(100, 100),
    batch_size=16,
    seed=522437
)
validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory=data_dir,
    color_mode='grayscale',
    validation_split=0.1,
    subset='validation',
    image_size=(100, 100),
    batch_size=16,
    seed=522437
)

In [ ]:
class_names = training_ds.class_names
num_classes = len(class_names)

class_names, num_classes

In [ ]:
training_ds = training_ds.cache().shuffle(1762).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

training_ds = training_ds.map(lambda x, y: (tf.divide(x, 255), y))
validation_ds = validation_ds.map(lambda x, y: (tf.divide(x, 255), y))

# Preview the data

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in training_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')

# Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(100, 100, 1)),
    tf.keras.layers.Conv2D(8, 5, activation='relu', kernel_initializer='variance_scaling'),
    tf.keras.layers.MaxPooling2D(strides=(2, 2)),
    tf.keras.layers.Conv2D(16, 5, activation='relu', kernel_initializer='variance_scaling'),
    tf.keras.layers.MaxPooling2D(strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=num_classes, activation='softmax', kernel_initializer='variance_scaling'),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(
    training_ds,
    validation_data=validation_ds,
    epochs=6
)

# Training and Validation Accuracy and Loss

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(6)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Predictions

In [ ]:
image = tf.keras.preprocessing.image.load_img(
    '/tmp/data/upper-case-m/1691665536687.png',
    target_size=(100, 100),
    color_mode='grayscale'
)
image_array = tf.keras.utils.img_to_array(image)
image_array = tf.expand_dims(image_array, 0)

predictions = model.predict(image_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

# Confusion Matrix

In [ ]:
test_images, test_labels = zip(*validation_ds.unbatch().as_numpy_iterator())
test_images = np.array(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

matrix = confusion_matrix(test_labels, predicted_labels, labels=range(num_classes))

display = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=[s[-1] for s in class_names])
display.plot(cmap=plt.cm.Blues, values_format='.4g')

plt.show()

In [ ]:
model.save('/tmp/models/model.keras');